# 基本的な線形回帰

このノートブックでは、線形回帰の基本的な実装と使用方法を学びます。

## 目次
1. [データの準備](#データの準備)
2. [基本的な実装](#基本的な実装)
3. [可視化と解釈](#可視化と解釈)
4. [予測と評価](#予測と評価)
5. [実践的な応用例](#実践的な応用例)
6. [演習問題](#演習問題)


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import seaborn as sns

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 乱数の固定
np.random.seed(42)

print("ライブラリのインポートが完了しました。")


## データの準備

実践的なデータセットを使用して線形回帰を学習します。


In [ ]:
# 住宅価格予測データの生成
np.random.seed(42)

# 特徴量の生成
n_samples = 200
area = np.random.uniform(30, 150, n_samples)  # 面積 (m²)
rooms = np.random.randint(1, 6, n_samples)  # 部屋数
age = np.random.uniform(0, 30, n_samples)   # 築年数
distance = np.random.uniform(0, 20, n_samples)  # 駅からの距離 (km)

# 価格の生成（線形関係 + ノイズ）
price = (50 * area + 100 * rooms - 5 * age - 10 * distance + 
         np.random.normal(0, 200, n_samples))

# データフレームの作成
data = pd.DataFrame({
    'area': area,
    'rooms': rooms,
    'age': age,
    'distance': distance,
    'price': price
})

print("データの基本情報:")
print(data.describe())
print(f"\nデータ形状: {data.shape}")

# データの可視化
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 各特徴量と価格の散布図
features = ['area', 'rooms', 'age', 'distance']
for i, feature in enumerate(features):
    row = i // 2
    col = i % 2
    axes[row, col].scatter(data[feature], data['price'], alpha=0.6)
    axes[row, col].set_xlabel(feature)
    axes[row, col].set_ylabel('価格 (万円)')
    axes[row, col].set_title(f'{feature} vs 価格')
    axes[row, col].grid(True, alpha=0.3)

# 相関行列
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[1, 2])
axes[1, 2].set_title('相関行列')

plt.tight_layout()
plt.show()

# 相関係数の確認
print("\n価格との相関係数:")
for feature in features:
    corr = data[feature].corr(data['price'])
    print(f"{feature}: {corr:.3f}")


## 基本的な実装

scikit-learnを使用して線形回帰を実装します。


In [ ]:
# データの分割
X = data[['area', 'rooms', 'age', 'distance']]
y = data['price']

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"訓練データ: {X_train.shape}")
print(f"テストデータ: {X_test.shape}")

# 線形回帰モデルの作成と訓練
lr = LinearRegression()
lr.fit(X_train, y_train)

# 回帰係数の表示
print(f"\n回帰係数:")
print(f"切片: {lr.intercept_:.2f}")
for feature, coef in zip(X.columns, lr.coef_):
    print(f"{feature}: {coef:.2f}")

# 回帰式の表示
equation = f"価格 = {lr.intercept_:.2f}"
for feature, coef in zip(X.columns, lr.coef_):
    equation += f" + {coef:.2f} × {feature}"
print(f"\n回帰式: {equation}")

# 係数の解釈
print(f"\n係数の解釈:")
print(f"面積が1m²増加すると、価格は{lr.coef_[0]:.2f}万円増加")
print(f"部屋数が1部屋増加すると、価格は{lr.coef_[1]:.2f}万円増加")
print(f"築年数が1年増加すると、価格は{lr.coef_[2]:.2f}万円減少")
print(f"駅からの距離が1km増加すると、価格は{lr.coef_[3]:.2f}万円減少")


## 可視化と解釈

回帰結果を可視化し、モデルの性能を理解します。


In [ ]:
# 予測値の計算
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

# 可視化
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. 実際値 vs 予測値（訓練データ）
axes[0, 0].scatter(y_train, y_train_pred, alpha=0.6, color='blue')
axes[0, 0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('実際の価格 (万円)')
axes[0, 0].set_ylabel('予測価格 (万円)')
axes[0, 0].set_title('訓練データ: 実際値 vs 予測値')
axes[0, 0].grid(True, alpha=0.3)

# 2. 実際値 vs 予測値（テストデータ）
axes[0, 1].scatter(y_test, y_test_pred, alpha=0.6, color='green')
axes[0, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0, 1].set_xlabel('実際の価格 (万円)')
axes[0, 1].set_ylabel('予測価格 (万円)')
axes[0, 1].set_title('テストデータ: 実際値 vs 予測値')
axes[0, 1].grid(True, alpha=0.3)

# 3. 残差プロット（訓練データ）
residuals_train = y_train - y_train_pred
axes[0, 2].scatter(y_train_pred, residuals_train, alpha=0.6, color='blue')
axes[0, 2].axhline(y=0, color='r', linestyle='--')
axes[0, 2].set_xlabel('予測価格 (万円)')
axes[0, 2].set_ylabel('残差 (万円)')
axes[0, 2].set_title('訓練データ: 残差プロット')
axes[0, 2].grid(True, alpha=0.3)

# 4. 残差プロット（テストデータ）
residuals_test = y_test - y_test_pred
axes[1, 0].scatter(y_test_pred, residuals_test, alpha=0.6, color='green')
axes[1, 0].axhline(y=0, color='r', linestyle='--')
axes[1, 0].set_xlabel('予測価格 (万円)')
axes[1, 0].set_ylabel('残差 (万円)')
axes[1, 0].set_title('テストデータ: 残差プロット')
axes[1, 0].grid(True, alpha=0.3)

# 5. 係数の可視化
feature_names = X.columns
coefficients = lr.coef_
colors = ['red' if coef < 0 else 'blue' for coef in coefficients]

axes[1, 1].barh(feature_names, coefficients, color=colors, alpha=0.7)
axes[1, 1].axvline(x=0, color='black', linestyle='-', alpha=0.3)
axes[1, 1].set_xlabel('回帰係数')
axes[1, 1].set_title('回帰係数の可視化')
axes[1, 1].grid(True, alpha=0.3)

# 6. 残差の分布
axes[1, 2].hist(residuals_test, bins=20, alpha=0.7, color='green', edgecolor='black')
axes[1, 2].axvline(x=0, color='r', linestyle='--')
axes[1, 2].set_xlabel('残差 (万円)')
axes[1, 2].set_ylabel('頻度')
axes[1, 2].set_title('テストデータ: 残差の分布')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 統計的検定（残差の正規性）
from scipy import stats
shapiro_stat, shapiro_p = stats.shapiro(residuals_test)
print(f"\n残差の正規性検定 (Shapiro-Wilk):")
print(f"統計量: {shapiro_stat:.4f}")
print(f"p値: {shapiro_p:.4f}")
if shapiro_p > 0.05:
    print("結論: 残差は正規分布に従います (p > 0.05)")
else:
    print("結論: 残差は正規分布に従いません (p ≤ 0.05)")


## 予測と評価

モデルの性能を定量的に評価します。


In [ ]:
# 評価指標の計算
def calculate_metrics(y_true, y_pred, dataset_name):
    """評価指標を計算する関数"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{dataset_name}の評価指標:")
    print(f"MSE (平均二乗誤差): {mse:.2f}")
    print(f"RMSE (平方根平均二乗誤差): {rmse:.2f}")
    print(f"MAE (平均絶対誤差): {mae:.2f}")
    print(f"R² (決定係数): {r2:.4f}")
    
    return mse, rmse, mae, r2

# 訓練データとテストデータの評価
train_metrics = calculate_metrics(y_train, y_train_pred, "訓練データ")
test_metrics = calculate_metrics(y_test, y_test_pred, "テストデータ")

# 過学習の確認
print(f"\n過学習の確認:")
print(f"訓練データ R²: {train_metrics[3]:.4f}")
print(f"テストデータ R²: {test_metrics[3]:.4f}")
print(f"差: {train_metrics[3] - test_metrics[3]:.4f}")

if train_metrics[3] - test_metrics[3] > 0.1:
    print("警告: 過学習の可能性があります")
else:
    print("良好: 過学習の兆候は見られません")

# 予測の信頼区間（簡易版）
def prediction_interval(y_true, y_pred, confidence=0.95):
    """予測の信頼区間を計算"""
    residuals = y_true - y_pred
    std_error = np.std(residuals)
    
    # 簡易的な信頼区間（正規分布を仮定）
    z_score = 1.96 if confidence == 0.95 else 2.576  # 95% or 99%
    margin = z_score * std_error
    
    return y_pred - margin, y_pred + margin

# 信頼区間の計算
lower_bound, upper_bound = prediction_interval(y_test, y_test_pred)

# 信頼区間の可視化
plt.figure(figsize=(12, 8))

# 実際値 vs 予測値（信頼区間付き）
plt.subplot(2, 2, 1)
plt.scatter(y_test, y_test_pred, alpha=0.6, color='green', label='データ点')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='完全予測')
plt.fill_between([y_test.min(), y_test.max()], 
                 [y_test.min() - 1.96 * np.std(residuals_test), 
                  y_test.max() - 1.96 * np.std(residuals_test)],
                 [y_test.min() + 1.96 * np.std(residuals_test), 
                  y_test.max() + 1.96 * np.std(residuals_test)],
                 alpha=0.2, color='gray', label='95%信頼区間')
plt.xlabel('実際の価格 (万円)')
plt.ylabel('予測価格 (万円)')
plt.title('予測精度（信頼区間付き）')
plt.legend()
plt.grid(True, alpha=0.3)

# 残差の分布
plt.subplot(2, 2, 2)
plt.hist(residuals_test, bins=20, alpha=0.7, color='green', edgecolor='black', density=True)
plt.axvline(x=0, color='r', linestyle='--', label='平均=0')
plt.axvline(x=np.mean(residuals_test), color='orange', linestyle='-', label=f'実際の平均={np.mean(residuals_test):.2f}')
plt.xlabel('残差 (万円)')
plt.ylabel('密度')
plt.title('残差の分布')
plt.legend()
plt.grid(True, alpha=0.3)

# 残差のQ-Qプロット
plt.subplot(2, 2, 3)
from scipy import stats
stats.probplot(residuals_test, dist="norm", plot=plt)
plt.title('残差のQ-Qプロット')
plt.grid(True, alpha=0.3)

# 特徴量の重要度
plt.subplot(2, 2, 4)
feature_importance = np.abs(lr.coef_)
feature_names = X.columns
colors = ['red' if coef < 0 else 'blue' for coef in lr.coef_]

bars = plt.barh(feature_names, feature_importance, color=colors, alpha=0.7)
plt.xlabel('係数の絶対値')
plt.title('特徴量の重要度')
plt.grid(True, alpha=0.3)

# 係数の値をバーに表示
for i, (bar, coef) in enumerate(zip(bars, lr.coef_)):
    plt.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2, 
             f'{coef:.2f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

# 予測例
print(f"\n予測例:")
print("=" * 50)
sample_idx = 0
sample_features = X_test.iloc[sample_idx]
sample_actual = y_test.iloc[sample_idx]
sample_pred = y_test_pred[sample_idx]

print(f"サンプル {sample_idx + 1}:")
for feature, value in sample_features.items():
    print(f"  {feature}: {value:.2f}")
print(f"  実際の価格: {sample_actual:.2f}万円")
print(f"  予測価格: {sample_pred:.2f}万円")
print(f"  誤差: {abs(sample_actual - sample_pred):.2f}万円")
print(f"  誤差率: {abs(sample_actual - sample_pred) / sample_actual * 100:.1f}%")


## 実践的な応用例

実際のビジネスシーンでの線形回帰の活用例を示します。


In [ ]:
# ビジネス応用例1: 売上予測
print("=== ビジネス応用例1: 売上予測 ===")

# 売上データの生成
np.random.seed(42)
months = 24
advertising = np.random.uniform(10, 100, months)  # 広告費（万円）
price = np.random.uniform(100, 500, months)  # 商品価格（円）
competition = np.random.uniform(0, 10, months)  # 競合他社数

# 売上の生成（線形関係 + ノイズ）
sales = (0.5 * advertising + 0.002 * price - 2 * competition + 
         np.random.normal(0, 10, months))

sales_data = pd.DataFrame({
    'advertising': advertising,
    'price': price,
    'competition': competition,
    'sales': sales
})

# 線形回帰モデルの構築
X_sales = sales_data[['advertising', 'price', 'competition']]
y_sales = sales_data['sales']

lr_sales = LinearRegression()
lr_sales.fit(X_sales, y_sales)

print(f"売上予測モデル:")
print(f"売上 = {lr_sales.intercept_:.2f} + {lr_sales.coef_[0]:.2f} × 広告費 + {lr_sales.coef_[1]:.2f} × 価格 + {lr_sales.coef_[2]:.2f} × 競合数")

# ビジネス解釈
print(f"\nビジネス解釈:")
print(f"広告費を1万円増加させると、売上は{lr_sales.coef_[0]:.2f}万円増加")
print(f"価格を1円上げると、売上は{lr_sales.coef_[1]:.2f}万円増加")
print(f"競合他社が1社増えると、売上は{lr_sales.coef_[2]:.2f}万円減少")

# 予測例
new_advertising = 50
new_price = 300
new_competition = 5
predicted_sales = lr_sales.predict([[new_advertising, new_price, new_competition]])[0]

print(f"\n予測例:")
print(f"広告費: {new_advertising}万円, 価格: {new_price}円, 競合: {new_competition}社")
print(f"予測売上: {predicted_sales:.2f}万円")


In [ ]:
# ビジネス応用例2: 顧客満足度分析
print("\n=== ビジネス応用例2: 顧客満足度分析 ===")

# 顧客満足度データの生成
np.random.seed(42)
n_customers = 100

# 特徴量
response_time = np.random.uniform(1, 10, n_customers)  # 応答時間（分）
service_quality = np.random.uniform(1, 10, n_customers)  # サービス品質（1-10）
price_satisfaction = np.random.uniform(1, 10, n_customers)  # 価格満足度（1-10）

# 満足度の生成
satisfaction = (8 - 0.3 * response_time + 0.4 * service_quality + 0.2 * price_satisfaction + 
                np.random.normal(0, 0.5, n_customers))

satisfaction_data = pd.DataFrame({
    'response_time': response_time,
    'service_quality': service_quality,
    'price_satisfaction': price_satisfaction,
    'satisfaction': satisfaction
})

# 線形回帰モデルの構築
X_satisfaction = satisfaction_data[['response_time', 'service_quality', 'price_satisfaction']]
y_satisfaction = satisfaction_data['satisfaction']

lr_satisfaction = LinearRegression()
lr_satisfaction.fit(X_satisfaction, y_satisfaction)

print(f"顧客満足度モデル:")
print(f"満足度 = {lr_satisfaction.intercept_:.2f} + {lr_satisfaction.coef_[0]:.2f} × 応答時間 + {lr_satisfaction.coef_[1]:.2f} × サービス品質 + {lr_satisfaction.coef_[2]:.2f} × 価格満足度")

# 改善提案
print(f"\n改善提案:")
if lr_satisfaction.coef_[0] < 0:
    print(f"応答時間を1分短縮すると、満足度は{abs(lr_satisfaction.coef_[0]):.2f}ポイント向上")
if lr_satisfaction.coef_[1] > 0:
    print(f"サービス品質を1ポイント向上させると、満足度は{lr_satisfaction.coef_[1]:.2f}ポイント向上")
if lr_satisfaction.coef_[2] > 0:
    print(f"価格満足度を1ポイント向上させると、満足度は{lr_satisfaction.coef_[2]:.2f}ポイント向上")

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

features = ['response_time', 'service_quality', 'price_satisfaction']
titles = ['応答時間', 'サービス品質', '価格満足度']
colors = ['red', 'green', 'blue']

for i, (feature, title, color) in enumerate(zip(features, titles, colors)):
    axes[i].scatter(satisfaction_data[feature], satisfaction_data['satisfaction'], 
                   alpha=0.6, color=color)
    axes[i].set_xlabel(title)
    axes[i].set_ylabel('満足度')
    axes[i].set_title(f'{title} vs 満足度')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 演習問題

### 問題1: 特徴量エンジニアリング
既存の特徴量から新しい特徴量を作成し、モデルの性能を改善してください。

### 問題2: 外れ値の処理
外れ値を検出し、適切に処理してモデルの性能を評価してください。

### 問題3: 交互作用項の追加
特徴量間の交互作用を考慮したモデルを作成してください。

### 問題4: モデルの解釈
回帰係数の統計的有意性を検定し、モデルの信頼性を評価してください。


## まとめ

このノートブックでは、線形回帰の基本的な実装と使用方法を学びました：

### 学習した内容
1. **データの準備**: 特徴量と目的変数の分割
2. **モデルの構築**: scikit-learnを使用した線形回帰
3. **可視化**: 回帰結果の解釈と診断
4. **評価**: 定量的な性能評価指標
5. **応用例**: 実践的なビジネスシーンでの活用

### 重要なポイント
- **データの分割**: 訓練データとテストデータの適切な分割
- **モデルの解釈**: 回帰係数の意味とビジネス解釈
- **性能評価**: 複数の評価指標による総合的な判断
- **可視化**: 残差分析によるモデルの診断

### 次のステップ
- [最急降下法の実装](./gradient_descent_implementation.ipynb)
- [正規方程式の実装](./normal_equation_implementation.ipynb)
- [特徴量スケーリング](../02_feature_scaling/02_feature_scaling.md)
- [統計的推論](../03_statistical_inference/03_statistical_inference.md)
